In [1]:
import pandas as pd
import numpy as np

### Development Indicators

Development indicators were obtained from three data sources—UNDP, the World Development Indicators (WDI), and Our World in Data. These sources differ in metadata structure, variable naming conventions, and data types. Each dataset was first cleaned and transformed into an analogous structure. The harmonized datasets were subsequently merged using ISO alpha-3 country codes and year identifiers.  

#### From our world in data

For datasets from our world in data datasets, the following steps are applied within a for loop:
1. Years outside the range of 1990 to 2020 are pruned out. 
2. The column name 'Code' is changed to 'iso_o', and 'Entity' is changed to 'country'

In [2]:
literacy = pd.read_csv('../data/raw/cross-country-literacy-rates.csv')
gini = pd.read_csv('../data/raw/economic-inequality-gini-index.csv')
institute = pd.read_csv('../data/raw/government-effectiveness-vs-electoral-democracy-index.csv')
corruption = pd.read_csv('../data/raw/political-corruption-index.csv')

In [3]:
owd = {'literacy': literacy,
        'gini': gini,
        'institute': institute,
        'corruption': corruption}

In [4]:
for series, data in owd.items():
    print(series, data['Year'].min(), data['Year'].max())

literacy 1475 2023
gini 1963 2024
institute 1789 2024
corruption 1789 2024


In [5]:
for series, data in owd.items():
    data = data[(data['Year'] >= 1990) & (data['Year'] <= 2020)]
    data = data.rename(columns={'Code': 'iso_o', 'Entity': 'country', 'Year':'year'})
    globals()[series] = data

In [6]:
literacy.head(2)

,country,iso_o,year,Literacy rate
2,Afghanistan,AFG,2011,31.00000
3,Afghanistan,AFG,2015,33.75384


In [7]:
print(literacy['iso_o'].nunique())
print(gini['iso_o'].nunique())
print(institute['iso_o'].nunique())
print(corruption['iso_o'].nunique())

159
168
181
181


In [8]:
merged_owd = literacy.merge(gini, on = ['country','iso_o','year'], how = 'left').\
            merge(institute, on = ['country','iso_o','year'], how = 'left').\
            merge(corruption, on = ['country','iso_o','year'], how = 'left')

In [9]:
merged_owd = merged_owd.rename(columns = {'Literacy rate': 'literacy_rate', 'World region according to OWID_x':'region', 
                                          'Electoral Democracy ndex':'electoral_dem_index', 'Government Effectiveness Index':'government_effec_index',
                                          'Political Corruption Index': 'political_corruption_index', 'Gini coefficient':'gini'})

In [10]:
merged_owd = merged_owd.drop(columns= ['World region according to OWID_y', 'World region according to OWID'])

In [11]:
merged_owd.head(2)

,country,iso_o,year,literacy_rate,gini,region,electoral_dem_index,government_effec_index,political_corruption_index
0,Afghanistan,AFG,2011,31.00000,NaN,NaN,0.358,-1.474,0.958
1,Afghanistan,AFG,2015,33.75384,NaN,NaN,0.353,-1.396,0.904


In [12]:
regional_entries = ['Central and Southern Asia (SDG)', 'East Asia and Pacific (WB)','Eastern and South-Eastern Asia (SDG)',
 'Europe and Central Asia (WB)','Europe and Northern America (SDG)','Latin America and Caribbean (WB)','Latin America and the Caribbean (SDG)',
 'Low-income countries','Lower-middle-income countries', 'Middle East and North Africa (WB)', 'Middle-income countries','North America (WB)',
 'North Korea','Northern Africa and Western Asia (SDG)','Oceania (excluding Australia and New Zealand) (SDG)','Saint Helena','South Asia (WB)',
 'Sub-Saharan Africa (SDG)','Sub-Saharan Africa (WB)', 'Upper-middle-income countries', 'World']

In [13]:
merged_owd = merged_owd[~merged_owd['country'].isin(regional_entries)]

In [14]:
merged_owd.isna().sum()

country                         0
iso_o                           0
year                            0
literacy_rate                   0
gini                          459
region                        459
electoral_dem_index            53
government_effec_index        152
political_corruption_index     53
dtype: int64

In [15]:
print(f"Earliest year is {merged_owd['year'].min()} and the latest year is {merged_owd['year'].max()}.")

Earliest year is 1990 and the latest year is 2020.


#### From WDI

For WDI datasets, the following steps are applied within a for loop:
1. The column name 'Country Code' is changed to 'iso_o'
2. All datasets are pruned from the row 266. 
3. Year column names are cleaned. 
4. Year column values are converted to numeric and rounded upto 3 decimal places. '..' is replaced by NaN. 
5. Meta data columns are dropped. 
6. Dataframes are stacked to long format.
7. Years other than the range from 1990 to 2020 are trimmed out.

In [16]:
GDP_pc = pd.read_csv('../data/raw/11606a24-87be-4d81-92b2-a318c1453d9d_Series - Metadata.csv')
mortality = pd.read_csv('../data/raw/1b88092e-3767-472c-943e-c094fe6c827b_Series - Metadata.csv')
primary_enroll = pd.read_csv('../data/raw/b6d35734-229f-4801-b42a-ef191a199de3_Series - Metadata.csv')
GDP_pc_gr = pd.read_csv('../data/raw/ef17b06f-2530-4235-9d0b-48728c3f92d9_Series - Metadata.csv')

In [17]:
GDP_pc.head(2)

,Series Name,Series Code,Country Name,Country Code,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,Afghanistan,AFG,..,..,..,..,..,..,...,575.146245808546,565.569730408751,563.872336723147,562.769574140988,553.125151688293,557.861533207459,527.834554499306,408.625855217403,377.665627080705,378.06630312259
1,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,Albania,ALB,1682.8810874018,1218.96427317362,1138.23773998809,1254.67888414888,1367.22058963712,1558.96638565461,...,4045.11934884933,4199.53912925398,4431.55635814168,4648.23078660658,4893.82375538717,5072.43648354739,4980.61699524696,5511.75743402862,5867.65096230557,6198.17195301204


In [18]:
wdi = {'GDP_pc': GDP_pc,
        'mortality': mortality,
        'primary_enroll': primary_enroll,
        'GDP_pc_gr': GDP_pc_gr}

In [19]:
for series, data in wdi.items():
    data = data.rename(columns = {'Country Code': 'iso_o', 'Country Name':'country'}).iloc[:265, :]
    data = data.drop(columns= ['Series Name', 'Series Code'])
    data.columns = data.columns.str.replace(r' \[YR\d+\]', '', regex=True)
    data.iloc[:, 2:] = data.iloc[:, 2:].apply(pd.to_numeric, errors='coerce').replace('..', np.nan).round(3)
    data_long = data.set_index(['iso_o', 'country']).stack().reset_index()
    data_long = data_long.rename(columns = {'level_2': 'year'})
    data_long['year'] = pd.to_numeric(data_long['year'], errors='coerce')
    data_long = data_long[(data_long['year'] >= 1990) & (data_long['year'] <= 2020)]
    wdi[series] = data_long
    globals()[series] = data_long

In [20]:
GDP_pc.head(2)

,iso_o,country,year,0
0,AFG,Afghanistan,2000,308.318
1,AFG,Afghanistan,2001,277.118


In [21]:
GDP_pc = GDP_pc.rename(columns = {0: 'GDP_pc'})
mortality = mortality.rename(columns = {0: 'mortality'})
primary_enroll= primary_enroll.rename(columns = {0: 'primary_enroll'})
GDP_pc_gr = GDP_pc_gr.rename(columns = {0: 'GDP_pc_gr'})

In [22]:
merged_wdi = GDP_pc.merge(mortality, on = ['iso_o','country', 'year'], how = 'left').merge(primary_enroll, on = ['iso_o', 'country', 'year'], how = 'left').\
            merge(GDP_pc_gr, on = ['iso_o', 'country', 'year'], how = 'left')

In [23]:
merged_wdi.isna().sum()

iso_o                0
country              0
year                 0
GDP_pc               0
mortality          399
primary_enroll    3533
GDP_pc_gr           40
dtype: int64

In [24]:
print(f"Earliest year is {merged_wdi['year'].min()} and the latest year is {merged_wdi['year'].max()}.")

Earliest year is 1990 and the latest year is 2020.


#### From UNDP

For UNDP datasets, the following steps are applied within a for loop:
1. The column name 'countryIsoCode' is changed to 'iso_o' 
2. Meta data columns are dropped. 
3. Years other than the range from 1990 to 2020 are trimmed out.

In [25]:
gii = pd.read_excel("../data/raw/hdr-data.xlsx")
lfp_f = pd.read_excel("../data/raw/hdr-data (1).xlsx")
parliament_seat_f = pd.read_excel("../data/raw/hdr-data (2).xlsx")
hdi = pd.read_excel("../data/raw/hdr-data (3).xlsx")
life_exp = pd.read_excel("../data/raw/hdr-data (4).xlsx")
mean_schooling = pd.read_excel("../data/raw/hdr-data (5).xlsx")
coef_human_inequality = pd.read_excel("../data/raw/hdr-data (6).xlsx")
inequality_edu = pd.read_excel("../data/raw/hdr-data (7).xlsx")
inequality_income = pd.read_excel("../data/raw/hdr-data (8).xlsx")
inequality_le = pd.read_excel("../data/raw/hdr-data (9).xlsx")

In [26]:
undp = {'gii': gii, 'lfp_f':lfp_f, 'parliament_seat_f':parliament_seat_f, 'hdi': hdi, 'life_exp':life_exp, 
        'mean_schooling':mean_schooling,'coef_human_inequality':coef_human_inequality, 'inequality_edu':inequality_edu, 
        'inequality_income':inequality_income, 'inequality_le':inequality_le}

In [27]:
for series, data in undp.items():
    data = data.rename(columns = {'countryIsoCode': 'iso_o'})
    data = data.drop(columns= ['indexCode', 'index', 'dimension', 'indicatorCode', 'indicator', 'yearStr', 'actualValue', 'note'])
    data = data[(data['year'] >= 1990) & (data['year'] <= 2020)]
    wdi[series] = data
    globals()[series] = data

In [28]:
gii = gii.rename(columns = {'value': 'gii'})
lfp_f = lfp_f.rename(columns = {'value': 'lfp_f'})
parliament_seat_f = parliament_seat_f.rename(columns = {'value': 'parliament_seat_f'})
hdi = hdi.rename(columns = {'value': 'hdi'})
life_exp = life_exp.rename(columns = {'value': 'life_exp'})
mean_schooling = mean_schooling.rename(columns = {'value': 'mean_schooling'})
coef_human_inequality = coef_human_inequality.rename(columns = {'value': 'coef_human_inequality'})
inequality_edu = inequality_edu.rename(columns = {'value': 'inequality_edu'})
inequality_income = inequality_income.rename(columns = {'value': 'inequality_income'})
inequality_le = inequality_le.rename(columns = {'value': 'inequality_le'})

In [29]:
inequality_le.head(2)

,iso_o,country,year,inequality_le
0,AFG,Afghanistan,2010,34.075
1,AFG,Afghanistan,2011,33.224


In [30]:
# merging the datasets
merged_undp = gii.merge(lfp_f, on = ['iso_o', 'country', 'year'], how='left').merge(parliament_seat_f, on = ['iso_o','country', 'year'], how='left').\
              merge(hdi, on = ['iso_o','country', 'year'], how='left').merge(life_exp, on = ['iso_o','country', 'year'], how='left').\
              merge(mean_schooling, on = ['iso_o','country', 'year'], how='left').merge(coef_human_inequality, on = ['iso_o','country', 'year'], how='left').\
              merge(inequality_edu, on = ['iso_o','country', 'year'], how='left').merge(inequality_income, on = ['iso_o','country', 'year'], how='left').\
              merge(inequality_le, on = ['iso_o','country', 'year'], how='left')

In [31]:
merged_undp.head(2)

,iso_o,country,year,gii,lfp_f,parliament_seat_f,hdi,life_exp,mean_schooling,coef_human_inequality,inequality_edu,inequality_income,inequality_le
0,AFG,Afghanistan,2008,0.690,43.79,25.872,0.446,59.708,1.743,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,2009,0.696,36.85,25.926,0.458,60.248,1.816,NaN,NaN,NaN,NaN


In [32]:
merged_undp.isna().sum()

iso_o                       0
country                     0
year                        0
gii                         0
lfp_f                       0
parliament_seat_f           0
hdi                        52
life_exp                    0
mean_schooling             18
coef_human_inequality    2582
inequality_edu           2461
inequality_income        2484
inequality_le            2321
dtype: int64

In [33]:
print(f"Earliest year is {merged_undp['year'].min()} and the latest year is {merged_undp['year'].max()}.")

Earliest year is 1990 and the latest year is 2020.


### Constructing final attributes dataset 

In [34]:
attributes = merged_wdi.merge(merged_owd, on = ['iso_o', 'country', 'year'], how='left').merge(merged_undp, on = ['iso_o', 'country', 'year'], how='left')

In [35]:
attributes.head(2)

,iso_o,country,year,GDP_pc,mortality,primary_enroll,GDP_pc_gr,literacy_rate,gini,region,...,gii,lfp_f,parliament_seat_f,hdi,life_exp,mean_schooling,coef_human_inequality,inequality_edu,inequality_income,inequality_le
0,AFG,Afghanistan,2000,308.318,110.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,2001,277.118,107.0,NaN,-10.119,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
print(attributes['country'].nunique())
print(attributes['iso_o'].nunique())

260
260


In [37]:
# Removing leading and trailing spaces
attributes[['country', 'iso_o']] = attributes[['country', 'iso_o']].apply(lambda x: x.str.strip())

In [38]:
regional_entries = ['Central and Southern Asia (SDG)', 'East Asia and Pacific (WB)','Eastern and South-Eastern Asia (SDG)',
 'Europe and Central Asia (WB)','Europe and Northern America (SDG)','Latin America and Caribbean (WB)','Latin America and the Caribbean (SDG)',
 'Low-income countries','Lower-middle-income countries', 'Middle East and North Africa (WB)', 'Middle-income countries','North America (WB)',
 'North Korea','Northern Africa and Western Asia (SDG)','Oceania (excluding Australia and New Zealand) (SDG)','Saint Helena','South Asia (WB)',
 'Sub-Saharan Africa (SDG)','Sub-Saharan Africa (WB)', 'Upper-middle-income countries', 'World']

In [39]:
attributes = attributes[~attributes['iso_o'].isin(regional_entries)]

In [40]:
print(attributes.index.min(), attributes.index.max())
print(attributes.shape)
# Seems the attribute dataset is in good shape

0 7742
(7743, 23)


### Migration data

In [41]:
mig = pd.read_csv("../data/raw/migration_imputed_RIKS_dec2021.csv")

In [42]:
mig.head(2)

,iso_or,origin,iso_des,destination,year,stock,flow,inflow,outflow
0,AAB,Antigua and Barbuda,ABW,Aruba,1960,16,NaN,NaN,NaN
1,AAB,Antigua and Barbuda,ABW,Aruba,1961,16,0.0,NaN,NaN


In [43]:
# Droping the metadata columns and out of study years
mig = mig.drop(columns = ['inflow','outflow'])
mig = mig[(mig['year'] >= 1990) & (mig['year'] <= 2020)]

In [44]:
print(mig['destination'].nunique())
print(mig['origin'].nunique())

237
240


In [45]:
print(set(mig['origin'].unique()) - set(mig['destination'].unique()))

{'Pitcairn Islands', 'Holy See', 'Sahrawi Arab Dem. Rep.'}


In [46]:
# The following three countries are in origin, but nobody ever migrated to these countries. Also, these are not in the attributes. 
extra = ['Holy See', 'Pitcairn Islands', 'Sahrawi Arab Dem. Rep.']

In [47]:
mig = mig[~mig['origin'].isin(extra)]

In [48]:
print(mig['destination'].nunique())
print(mig['origin'].nunique())

237
237


In [49]:
extra_in_migration = list(set(mig['destination'].unique()) - set(attributes['country'].unique()))
extra_in_attributes = list(set(attributes['country'].unique()) - set(mig['destination'].unique()))

In [50]:
len(extra_in_attributes)

75

In [51]:
len(extra_in_migration)

52

In [52]:
mig[mig['destination']== "Korea; Rep."].head(2)

,iso_or,origin,iso_des,destination,year,stock,flow
9590,AAB,Antigua and Barbuda,ROK,Korea; Rep.,1990,0,0.0
9591,AAB,Antigua and Barbuda,ROK,Korea; Rep.,1991,0,0.0


In [53]:
mig[mig['destination']== "Korea; Dem. People's Rep."].head(2)

,iso_or,origin,iso_des,destination,year,stock,flow
9313,AAB,Antigua and Barbuda,PRK,Korea; Dem. People's Rep.,1990,0,0.0
9314,AAB,Antigua and Barbuda,PRK,Korea; Dem. People's Rep.,1991,0,0.0


In [54]:
#attributes[attributes['country'] == "Korea, Rep."]

This list has countries mentioned differently in the migration data and the regional entries. As the ISO codes are different in migration data from Alpha3 codes, and the number of different entries are more in country codes, than in  country names, I am using country names to merge the datasets. Hence, I am renaming the countries in the attribute dataframe, following the entries in the migration dataframe.

In [55]:
#renaming the countries based on the migration data
attributes.loc[attributes['country'] == 'Brunei', 'country'] = 'Brunei Darussalam'
attributes.loc[attributes['country'] == 'Cape Verde', 'country'] = 'Cabo Verde'
attributes.loc[attributes['country'] == 'Central African Republic', 'country'] = 'Central African Rep.'
attributes.loc[attributes['country'] == 'Congo', 'country'] = 'Congo; Rep.'
attributes.loc[attributes['country'] == 'Congo, Rep.', 'country'] = 'Congo; Rep.'
attributes.loc[attributes['country'] == 'Dominican Republic', 'country'] = 'Dominican Rep.'
attributes.loc[attributes['country'] == 'East Timor', 'country'] = 'Timor-Leste'
attributes.loc[attributes['country'] == 'Egypt', 'country'] = 'Egypt; Arab Rep.'
attributes.loc[attributes['country'] == 'Egypt, Arab Rep.', 'country'] = 'Egypt; Arab Rep.'
attributes.loc[attributes['country'] == 'East Timor', 'country'] = 'Timor-Leste'
attributes.loc[attributes['country'] == 'Eswatini', 'country'] = 'Swaziland'
attributes.loc[attributes['country'] == 'Gambia', 'country'] = 'Gambia; The'
attributes.loc[attributes['country'] == 'Gambia, The', 'country'] = 'Gambia; The'
attributes.loc[attributes['country'] == 'Iran', 'country'] = 'Iran; Islamic Rep.'
attributes.loc[attributes['country'] == 'Iran, Islamic Rep.', 'country'] = 'Iran; Islamic Rep.'
attributes.loc[attributes['country'] == 'Kyrgyzstan', 'country'] = 'Kyrgyz Rep.'
attributes.loc[attributes['country'] == 'Kyrgyz Republic', 'country'] = 'Kyrgyz Rep.'
attributes.loc[attributes['country'] == 'Laos', 'country'] = "Lao People's Dem. Rep."
attributes.loc[attributes['country'] == 'Lao PDR', 'country'] = "Lao People's Dem. Rep."
attributes.loc[attributes['country'] == 'Macao', 'country'] = 'Macao SAR; China'
attributes.loc[attributes['country'] == 'Macao SAR, China', 'country'] = 'Macao SAR; China'
attributes.loc[attributes['country'] == 'Russia', 'country'] = 'Russian Federation'
attributes.loc[attributes['country'] == 'Korea, Rep.', 'country'] = "Korea; Rep."
attributes.loc[attributes['country'] == 'Syria', 'country'] = 'Syrian Arab Rep.'
attributes.loc[attributes['country'] == 'Syrian Arab Republic', 'country'] = 'Syrian Arab Rep.'
attributes.loc[attributes['country'] == 'Palestine', 'country'] = 'West Bank and Gaza'
attributes.loc[attributes['country'] == 'Yemen', 'country'] = 'Yemen; Rep.'
attributes.loc[attributes['country'] == 'Yemen, Rep.', 'country'] = 'Yemen; Rep.'
attributes.loc[attributes['country'] == 'Democratic Republic of Congo', 'country'] = "Congo; Dem. Rep."
attributes.loc[attributes['country'] == 'Congo, Dem. Rep.', 'country'] = "Congo; Dem. Rep."
attributes.loc[attributes['country'] == 'Turkiye', 'country'] = "Turkey"
attributes.loc[attributes['country'] == 'United States', 'country'] = "United States of America"
attributes.loc[attributes['country'] == 'Netherlands', 'country'] = "Netherlands Antilles"
attributes.loc[attributes['country'] == 'Czechia', 'country'] = "Czech Rep."
attributes.loc[attributes['country'] == 'Viet Nam', 'country'] = "Vietnam"
attributes.loc[attributes['country'] == 'Venezuela, RB', 'country'] = "Venezuela"
attributes.loc[attributes['country'] == 'Puerto Rico (US)', 'country'] = "Puerto Rico"
attributes.loc[attributes['country'] == 'Hong Kong SAR, China', 'country'] = "Hong Kong; SAR China"
attributes.loc[attributes['country'] == 'Micronesia, Fed. Sts.', 'country'] = "Micronesia; Fed. Sts."
attributes.loc[attributes['country'] == 'Slovak Republic', 'country'] = "Slovak Rep."
attributes.loc[attributes['country'] == 'Somalia, Fed. Rep.', 'country'] = "Somalia"
attributes.loc[attributes['country'] == 'Bahamas, The', 'country'] = "Bahamas; The"

After renaming these countries, there remains 26 more elements in the list `extra_in_attributes` those are regional entries. I am removing those from the attributes data

In [56]:
extra_in_migration1 = list(set(mig['destination'].unique()) - set(attributes['country'].unique()))
extra_in_attributes1 = list(set(attributes['country'].unique()) - set(mig['destination'].unique()))

In [57]:
attributes = attributes[~attributes['country'].isin(extra_in_attributes1)]

In [58]:
attributes['country'].nunique()

211

Except Yemen and Netherlands, all other elements of `extra_in_migration1` are countries those got splitted in the last couple of decades into more than one nations. It also contains some islands and North Korea those are absent in the attributes. Hence. I am removing these entries from the migration dataframe.

In [59]:
mig.loc[mig['origin'] == 'Netherlands; The', 'origin'] = 'Netherlands Antilles'
mig.loc[mig['origin'] == 'Yemen Arab Rep.', 'origin'] = 'Yemen; Rep.'
mig.loc[mig['destination'] == 'Netherlands; The', 'destination'] = 'Netherlands Antilles'
mig.loc[mig['destination'] == 'Yemen Arab Rep.', 'destination'] = 'Yemen; Rep.'

In [60]:
extra_in_migration2 = list(set(mig['destination'].unique()) - set(attributes['country'].unique()))
extra_in_attributes2 = list(set(attributes['country'].unique()) - set(mig['destination'].unique()))

In [61]:
len(extra_in_migration2)

24

In [62]:
len(extra_in_attributes2)

0

In [63]:
mig = mig[~mig['origin'].isin(extra_in_migration2)]
mig = mig[~mig['destination'].isin(extra_in_migration2)]

In [64]:
print(mig['origin'].nunique())
print(mig['destination'].nunique())

211
211


In [65]:
# Removing self loops
mig = mig[mig['origin'] != mig['destination']]
mig.shape

(1229093, 7)

In [66]:
# Checking for duplicates and removing if any
mig_duplicate = mig.duplicated(subset=['origin', 'destination', 'year'], keep = 'last')
mig_unique = mig[~mig_duplicate]
mig_unique.shape 

(1224645, 7)

In [67]:
print(mig_unique.groupby(['origin', 'destination']).ngroups)
# So, there are 43918 country pairs

43918


In [68]:
mig_unique = mig_unique.reset_index(drop = True)

In [69]:
mig_unique.head(2)

,iso_or,origin,iso_des,destination,year,stock,flow
0,AAB,Antigua and Barbuda,ABW,Aruba,1990,32,2.0
1,AAB,Antigua and Barbuda,ABW,Aruba,1991,34,2.0


In [70]:
mig_unique['origin'].nunique()

211

In [71]:
mig_unique['destination'].nunique()

211

In [72]:
attributes['country'].nunique()

211

In [73]:
assert set(mig_unique['origin'].unique()) == set(mig_unique['destination'].unique()) == set(attributes['country'].unique())

Now, both the bilateral migration data and the attributes data have exactly the same 211 countries. 

In [74]:
mig_unique.to_csv('../data/cleaned/bilateral_migration.csv', encoding = 'utf-8', index=False)

In [75]:
attributes.to_csv('../data/cleaned/attributes.csv', encoding = 'utf-8', index=False)

### Constructing bilateral panel data

Steps to construct the panel dataset:
1. Generating a list of countries and a list of years
2. Constructing a balanced panel by creating all possible country–pair combinations for each year in the study period
3. Merging the blank panel with bilateral migration data using origin–destination–year identifiers
4. Creating a partner-attributes dataset by duplicating the country attributes data, appending a '_d' suffix to all variables, and renaming the 'origin' identifier to 'destination'
5. Merging destination-country attributes into the panel dataset
6. Computing absolute differences between origin and destination country development indicators

In [76]:
# Making country list and year list
country_list = mig_unique['origin'].unique()
print(len(country_list))
years = list(range(1990, 2021))

211


In [77]:
# Construcing black matrix
blank = pd.MultiIndex.from_product([country_list, country_list, years], names=['origin', 'destination', 'year'])
blank_panel = blank.to_frame(index = False)
blank_panel = blank_panel[blank_panel['origin'] != blank_panel['destination']]
print(blank_panel.shape)

(1373610, 3)


In [78]:
(211*211-211)*31

1373610

In [79]:
blank_panel.head(1)

,origin,destination,year
31,Antigua and Barbuda,Aruba,1990


In [80]:
# Merging migration data with the blank matrix
migration = blank_panel.merge(mig_unique, on = ['origin', 'destination', 'year'], how = 'left')

In [81]:
migration.head(2)

,origin,destination,year,iso_or,iso_des,stock,flow
0,Antigua and Barbuda,Aruba,1990,AAB,ABW,32.0,2.0
1,Antigua and Barbuda,Aruba,1991,AAB,ABW,34.0,2.0


In [82]:
assert migration.shape[0] == blank_panel.shape[0] 

In [83]:
# Merging the attributes data with the bilateral migration data on origin and year columns
attributes = attributes.rename(columns = {'country': 'origin'})
panel = migration.merge(attributes, on = ['origin', 'year'], how = 'left')
panel = panel.drop(columns = ['iso_or', 'iso_des'])

In [84]:
assert migration.shape[0] == blank_panel.shape[0] == panel.shape[0]

In [85]:
# Constructing a dataframe for the attributes of destination countries
partner_attributes = attributes.copy()

In [86]:
len(list(partner_attributes.columns))

23

In [87]:
partner_attributes = partner_attributes.drop(columns = ['iso_o'])

In [88]:
destination_attributes = {'origin': 'destination', 'GDP_pc': 'GDP_pc_d', 'mortality': 'mortality_d', 'primary_enroll': 'primary_enroll_d','GDP_pc_gr': 'GDP_pc_gr_d',
 'literacy_rate': 'literacy_rate_d', 'gini': 'gini_d', 'region': 'region_d', 'electoral_dem_index': 'electoral_dem_index_d','government_effec_index': 'government_effec_index_d',
'political_corruption_index': 'political_corruption_index_d', 'gii': 'gii_d', 'lfp_f': 'lfp_f_d', 'parliament_seat_f': 'parliament_seat_f_d','hdi': 'hdi_d',
'life_exp': 'life_exp_d','mean_schooling': 'mean_schooling_d', 'coef_human_inequality': 'coef_human_inequality_d', 'inequality_edu': 'inequality_edu_d',
'inequality_income': 'inequality_income_d','inequality_le': 'inequality_le_d'}

In [89]:
partner_attributes = partner_attributes.rename(columns = destination_attributes)

In [90]:
# Merging the partner attributes data with the bilateral migration data on destination and year columns
panel2 = panel.merge(partner_attributes, on = ['destination', 'year'], how = 'left')

In [91]:
len(list(panel2.columns))

46

In [92]:
panel2 = panel2[['origin', 'destination', 'year', 'stock', 'flow', 'iso_o', 'region', 'region_d', 'GDP_pc', 'GDP_pc_d','mortality','mortality_d','primary_enroll',
        'primary_enroll_d','GDP_pc_gr','GDP_pc_gr_d','literacy_rate','literacy_rate_d', 'gini','gini_d','electoral_dem_index','electoral_dem_index_d',
        'government_effec_index','government_effec_index_d','political_corruption_index','political_corruption_index_d','gii','gii_d','lfp_f','lfp_f_d',
        'parliament_seat_f', 'parliament_seat_f_d','hdi','hdi_d','life_exp','life_exp_d','mean_schooling','mean_schooling_d','coef_human_inequality',
        'coef_human_inequality_d','inequality_edu', 'inequality_edu_d', 'inequality_income','inequality_income_d','inequality_le','inequality_le_d']]

In [93]:
panel2.columns

Index(['origin', 'destination', 'year', 'stock', 'flow', 'iso_o', 'region',
       'region_d', 'GDP_pc', 'GDP_pc_d', 'mortality', 'mortality_d',
       'primary_enroll', 'primary_enroll_d', 'GDP_pc_gr', 'GDP_pc_gr_d',
       'literacy_rate', 'literacy_rate_d', 'gini', 'gini_d',
       'electoral_dem_index', 'electoral_dem_index_d',
       'government_effec_index', 'government_effec_index_d',
       'political_corruption_index', 'political_corruption_index_d', 'gii',
       'gii_d', 'lfp_f', 'lfp_f_d', 'parliament_seat_f', 'parliament_seat_f_d',
       'hdi', 'hdi_d', 'life_exp', 'life_exp_d', 'mean_schooling',
       'mean_schooling_d', 'coef_human_inequality', 'coef_human_inequality_d',
       'inequality_edu', 'inequality_edu_d', 'inequality_income',
       'inequality_income_d', 'inequality_le', 'inequality_le_d'],
      dtype='object')

In [94]:
panel2[['GDP_pc', 'GDP_pc_d', 'mortality', 'mortality_d', 'primary_enroll', 'primary_enroll_d','GDP_pc_gr', 'GDP_pc_gr_d']] = panel2[['GDP_pc', 
'GDP_pc_d', 'mortality', 'mortality_d', 'primary_enroll', 'primary_enroll_d','GDP_pc_gr', 'GDP_pc_gr_d']].apply(pd.to_numeric, errors='coerce')

In [95]:
# Constructing column for the absolutes difference between the origin and destination countries different development indicators
panel2['diff_GDP_pc'] = (panel2['GDP_pc'] - panel2['GDP_pc_d']).abs()
panel2['diff_mortality'] = (panel2['mortality'] - panel2['mortality_d']).abs()
panel2['diff_primary_enroll'] = (panel2['primary_enroll'] - panel2['primary_enroll_d']).abs()
panel2['diff_GDP_pc_gr'] = (panel2['GDP_pc_gr'] - panel2['GDP_pc_gr_d']).abs()
panel2['diff_literacy_rate'] = (panel2['literacy_rate'] - panel2['literacy_rate_d']).abs()
panel2['diff_gini'] = (panel2['gini'] - panel2['gini_d']).abs()
panel2['diff_elec_democracy'] = (panel2['electoral_dem_index'] - panel2['electoral_dem_index_d']).abs()
panel2['diff_govt_effect'] = (panel2['government_effec_index'] - panel2['government_effec_index_d']).abs()
panel2['diff_political_corruption'] = (panel2['political_corruption_index'] - panel2['political_corruption_index_d']).abs()
panel2['diff_gii'] = (panel2['gii'] - panel2['gii_d']).abs()
panel2['diff_lfp_f'] = (panel2['lfp_f'] - panel2['lfp_f_d']).abs()
panel2['diff_parliament_seat_f'] = (panel2['parliament_seat_f'] - panel2['parliament_seat_f_d']).abs()
panel2['diff_hdi'] = (panel2['hdi'] - panel2['hdi_d']).abs()
panel2['diff_life_exp'] = (panel2['life_exp'] - panel2['life_exp_d']).abs()
panel2['diff_mean_schooling'] = (panel2['mean_schooling'] - panel2['mean_schooling_d']).abs()
panel2['diff_coef_human_inequ'] = (panel2['coef_human_inequality'] - panel2['coef_human_inequality_d']).abs()
panel2['diff_inequality_edu'] = (panel2['inequality_edu'] - panel2['inequality_edu_d']).abs()
panel2['diff_inequality_income'] = (panel2['inequality_income'] - panel2['inequality_income_d']).abs()
panel2['diff_inequality_le'] = (panel2['inequality_le'] - panel2['inequality_le_d']).abs()

In [96]:
print(panel2.groupby(['origin', 'destination']).ngroups)
print(panel2['year'].nunique())
print(panel2.shape[0])

44310
31
1373610


In [97]:
211*211-211

44310

In [98]:
(211*211-211)*31

1373610

In [99]:
# constructing a dyad_id for all unique pair of countries
panel2['dyad_id'] = panel2['origin'] + '-' + panel2['destination'] 

In [100]:
panel2.to_csv('../data/cleaned/panel_migration.csv', encoding = 'utf-8', index=False)